In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt
import glob

In [2]:
home = Path.home()
today = dt.date.today()

In [3]:
sourcePath = Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新','Project team','ESC')
fileList = glob.glob(str(Path(sourcePath, 'ZRTR_US5A*xlsx')))
PNFVpath = Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新','PN FV description mapping table_ALL.xlsx')
PNFVfile = pd.read_excel(PNFVpath)

In [4]:
PNFVfile = PNFVfile[['Commodity', 'PN', 'Descr']]

In [5]:
for _ in fileList:
    print(_)

C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\ESC\ZRTR_US5A_0804.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\ESC\ZRTR_US5A_0906.xlsx
C:\Users\lulo\HP Inc\GPSTW SOP - 2021 日新\Project team\ESC\ZRTR_US5A_1006.xlsx


In [6]:
fileList[-1]

'C:\\Users\\lulo\\HP Inc\\GPSTW SOP - 2021 日新\\Project team\\ESC\\ZRTR_US5A_1006.xlsx'

In [7]:
totalESCList = []


In [8]:
for _ in fileList:
    print(_.split('\\')[-1])
    temp = pd.read_excel(_)
    tempdate = _.split('\\')[-1][10:14]
    tempdate = dt.datetime.strptime(tempdate, '%m%d').replace(year = today.year)
    try:
        temp = temp[['Material', 'Comp code curr(legal)']]
        temp = temp.rename(columns = {'Comp code curr(legal)' : 'Price', 'Material' : 'PN'})
    except:
        temp = temp[['Material', 'Standard price1']]
        temp = temp.rename(columns = {'Standard price1' : 'Price', 'Material' : 'PN'})

    temp.loc[:, 'date'] = tempdate
    temp = temp.merge(PNFVfile, on = 'PN', how = 'left')
    temp = temp.reset_index(drop = True)

    totalESCList.append(temp)

ZRTR_US5A_0804.xlsx
ZRTR_US5A_0906.xlsx
ZRTR_US5A_1006.xlsx


In [9]:
totalESC = pd.concat(totalESCList)


In [11]:
writer = pd.ExcelWriter(Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新','Project team','ESC', 'ESCtotal.xlsx'), datetime_format = 'yyyy-mm-dd')
totalESC.to_excel(writer, index = False)
writer.close()

In [12]:
totalESC_1 = totalESC[totalESC['Descr'].notna()]

In [13]:
totalESC_1_FV = totalESC_1.drop(columns=['PN', 'Commodity']).groupby(['Descr', 'date']).agg({'Price' : 'mean'})

In [14]:
totalESC_1_FV = totalESC_1_FV.reset_index()

In [15]:
totalESC_1 = totalESC_1.merge(totalESC_1_FV, on = ['Descr', 'date'], suffixes = ['', '_mean'])

In [16]:
totalESC_1

,PN,Price,date,Commodity,Descr,Price_mean
0,100613-021,1.57,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium UK,1.57
1,100614-016,1.5,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium EURO,1.50
2,100661-001,1.6,2022-08-04,POWERCORDS,PWC OPT-901 3-COND 1.83m C13,1.60
3,100661-021,1.34,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium AUST,1.34
4,109197-008,1.08,2022-08-04,POWERCORDS,PWC OPT-921 3-COND 1.83m C13,1.08
...,...,...,...,...,...,...
12961,N26505-001,14.8,2022-10-06,eMMC,UFS 128GB v3.0,14.80
12962,N26504-001,28,2022-10-06,eMMC,UFS 256GB v3.0,28.00
12963,N26506-001,28,2022-10-06,eMMC,UFS 256GB v3.0,28.00
12964,N26951-N01,49,2022-10-06,CHIPSET,Chipset C741,49.00


In [20]:
totalESC_1['Price'] = totalESC_1['Price'].astype(float).round(2)
totalESC_1['Price_mean'] = totalESC_1['Price_mean'].astype(float).round(2)

In [21]:
totalESC_1

,PN,Price,date,Commodity,Descr,Price_mean
0,100613-021,1.57,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium UK,1.57
1,100614-016,1.50,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium EURO,1.50
2,100661-001,1.60,2022-08-04,POWERCORDS,PWC OPT-901 3-COND 1.83m C13,1.60
3,100661-021,1.34,2022-08-04,POWERCORDS,PWC C13 1.0m tag Premium AUST,1.34
4,109197-008,1.08,2022-08-04,POWERCORDS,PWC OPT-921 3-COND 1.83m C13,1.08
...,...,...,...,...,...,...
12961,N26505-001,14.80,2022-10-06,eMMC,UFS 128GB v3.0,14.80
12962,N26504-001,28.00,2022-10-06,eMMC,UFS 256GB v3.0,28.00
12963,N26506-001,28.00,2022-10-06,eMMC,UFS 256GB v3.0,28.00
12964,N26951-N01,49.00,2022-10-06,CHIPSET,Chipset C741,49.00


In [22]:
writer = pd.ExcelWriter(Path(home, 'HP Inc', 'GPSTW SOP - 2021 日新','Project team','ESC', 'ESCtotal_averagePrice.xlsx'), datetime_format = 'yyyy-mm-dd')
totalESC_1.to_excel(writer, index = False)
writer.close()

In [19]:
totalESC_1[totalESC_1['Price_x'] != totalESC_1['Price_y']].reset_index(drop= True)

KeyError: 'Price_x'